## Import libraries

In [146]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from datetime import date, datetime

In [2]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
!pwd

/home/xavi/Documents/CreditRisk/notebooks


## Import data

In [76]:
CURRENT_DATE = date(2017, 12, 1)

In [4]:
folder: Path = Path('/home/xavi/Documents/CreditRisk/data/')

In [5]:
df: pd.DataFrame = pd.read_csv(folder / 'raw/loan_data_2007_2014.csv', nrows=10_000)

In [6]:
df.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077501,Borrower added on 12/22/11 > I need to upgrade my business technologies.<br>,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-15,171.62,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077430,Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to have it paid off as soon as possible/when i sell my old bike. I only need this money because the deal im looking at is to good to pass up.<br><br> Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to have it paid off as soon as possible/when i sell my old bike.I only need this money because the deal im looking at is to good to pass up. I have finished college with an associates degree in business and its takingmeplaces<br>,car,bike,309xx,GA,1.00,0.0,Apr-99,5.0,NaN,NaN,3.0,0.0,1687,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077175,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-01,2.0,NaN,NaN,2.0,0.0,2956,98.5,10.0,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-14,649.91,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1076863,"Borrower added on 12/21/11 > to pay for property tax (borrow from friend, need to pay back) & central A/C need to be replace. I'm very sorry to let my loan expired last time.<br>",other,personel,917xx,CA,20.00,0.0,Feb-96,1.0,35.0,NaN,10.0,0.0,5598,21.0,37.0,f,0.0,0.0,12226.302210,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-15,357.48,NaN,Jan-15,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-11,Current,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1075358,"Borrower added on 12/21/11 > I plan on combining three large interest bills together and f

In [7]:
len(df)

10000

In [8]:
len(df['id'].unique())

10000

In [9]:
df = df.set_index('id')

In [10]:
schema: pd.DataFrame = pd.read_json(folder / 'curated/data_dictionary.json', orient='records')

In [11]:
schema.head()

,LoanStatNew,Description
0,addr_state,The state provided by the borrower in the loan application
1,annual_inc,The self-reported annual income provided by the borrower during registration.
2,annual_inc_joint,The combined self-reported annual income provided by the co-borrowers during registration
3,application_type,Indicates whether the loan is an individual application or a joint application with two co-borrowers
4,collection_recovery_fee,post charge off collection fee


## Check data types

In [12]:
df.dtypes

Unnamed: 0                       int64
member_id                        int64
loan_amnt                        int64
funded_amnt                      int64
funded_amnt_inv                float64
term                            object
int_rate                       float64
installment                    float64
grade                           object
sub_grade                       object
emp_title                       object
emp_length                      object
home_ownership                  object
annual_inc                     float64
verification_status             object
issue_d                         object
loan_status                     object
pymnt_plan                      object
url                             object
desc                            object
purpose                         object
title                           object
zip_code                        object
addr_state                      object
dti                            float64
delinq_2yrs              

In [20]:
data_dict: pd.DataFrame = pd.merge(schema, df.dtypes.rename('current_type'), left_on='LoanStatNew', right_index=True, how='outer', indicator=True)

In [21]:
data_dict['target_type'] = pd.Series([], dtype='object')

In [25]:
data_dict['changed'] = pd.Series([False] * len(data_dict), dtype='bool')

In [27]:
data_dict['_merge'] = data_dict['_merge'].replace({'both': 'both', 'left_only': 'schema', 'right_only': 'data'})

In [28]:
data_dict.sort_values(by='current_type')

,LoanStatNew,Description,current_type,_merge,target_type,changed
31.0,loan_amnt,"The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.",int64,both,NaN,False
NaN,Unnamed: 0,NaN,int64,data,NaN,NaN
46.0,revol_bal,Total credit revolving balance,int64,both,NaN,False
15.0,funded_amnt,The total amount committed to that loan at that point in time.,int64,both,NaN,False
33.0,member_id,A unique LC assigned Id for the borrower member.,int64,both,NaN,False
41.0,policy_code,publicly available policy_code=1\nnew products not publicly available policy_code=2,int64,both,NaN,False
60.0,open_acc_6m,Number of open trades in last 6 months,float64,both,NaN,False
56.0,total_rec_prncp,Principal received to date,float64,both,NaN,False
55.0,total_rec_late_fee,Late fees received to date,float64,both,NaN,False
54.0,total_rec_int,Interest received to date,float64,both,NaN,False


Drop Column:

In [31]:
df = df.drop('Unnamed: 0', axis=1)

In [32]:
df['verification_status'].value_counts()

verification_status
Verified           3881
Source Verified    3069
Not Verified       3050
Name: count, dtype: int64

In [33]:
data_dict.loc[data_dict['LoanStatNew'] == 'verification_status', 'target_type'] = 'categorical'

In [57]:
data_dict.loc[data_dict['LoanStatNew'] == 'verification_status', 'changed'] = False

In [39]:
df['verification_status_joint'].value_counts()

Series([], Name: count, dtype: int64)

In [40]:
df['total_rev_hi_lim'].value_counts()

Series([], Name: count, dtype: int64)

In [43]:
df['purpose'].value_counts()

purpose
debt_consolidation    5155
credit_card           1503
other                  865
home_improvement       630
small_business         439
major_purchase         365
car                    298
wedding                217
medical                176
moving                 126
house                  113
vacation                91
renewable_energy        22
Name: count, dtype: int64

In [44]:
data_dict.loc[data_dict['LoanStatNew'] == 'purpose', 'target_type'] = 'categorical'

In [45]:
df['next_pymnt_d'].head()

id
1077501       NaN
1077430       NaN
1077175       NaN
1076863       NaN
1075358    Feb-16
Name: next_pymnt_d, dtype: object

In [46]:
data_dict.loc[data_dict['LoanStatNew'] == 'next_pymnt_d', 'target_type'] = 'date'

In [47]:
df['last_credit_pull_d'].head()

id
1077501    Jan-16
1077430    Sep-13
1077175    Jan-16
1076863    Jan-15
1075358    Jan-16
Name: last_credit_pull_d, dtype: object

In [48]:
data_dict.loc[data_dict['LoanStatNew'] == 'last_credit_pull_d', 'target_type'] = 'date'

In [53]:
data_dict['sample'] = data_dict['LoanStatNew']\
                        .apply(lambda v: df[v].head(30).to_list() if v in df.columns.values else np.nan)

In [55]:
data_dict.sort_values(by='_merge')

,LoanStatNew,Description,current_type,_merge,target_type,changed,sample
27.0,last_fico_range_high,The upper boundary range the borrower’s last FICO pulled belongs to.,NaN,schema,NaN,False,NaN
28.0,last_fico_range_low,The lower boundary range the borrower’s last FICO pulled belongs to.,NaN,schema,NaN,False,NaN
58.0,verified_status_joint,"Indicates if the co-borrowers' joint income was verified by LC, not verified, or if the income source was verified",NaN,schema,NaN,False,NaN
79.0,None,* Employer Title replaces Employer Name for all loans listed after 9/23/2013,NaN,schema,NaN,False,NaN
78.0,None,None,NaN,schema,NaN,False,NaN
19.0,id,A unique LC assigned ID for the loan listing.,NaN,schema,NaN,False,NaN
14.0,fico_range_low,The lower boundary range the borrower’s FICO at loan origination belongs to.,NaN,schema,NaN,False,NaN
13.0,fico_range_high,The upper boundary range the borrower’s FICO at loan origination belongs to.,NaN,schema,NaN,False,NaN
24.0,is_inc_v,"Indicates if income was verified by LC, not verified, or if the income source was verified",NaN,schema,NaN,False,NaN
71.0,total_rev_hi_lim,Total revolving high credit/credit limit,NaN,schema,NaN,False,NaN


In [65]:
for col in filter(lambda c: re.search('_d$', c) , df.columns.values):
    data_dict.loc[data_dict['LoanStatNew'] == col, 'target_type'] = 'date'

In [113]:
data_dict.sort_values(by='_merge')

,LoanStatNew,Description,current_type,_merge,target_type,changed,sample
27.0,last_fico_range_high,The upper boundary range the borrower’s last FICO pulled belongs to.,NaN,schema,NaN,False,NaN
28.0,last_fico_range_low,The lower boundary range the borrower’s last FICO pulled belongs to.,NaN,schema,NaN,False,NaN
58.0,verified_status_joint,"Indicates if the co-borrowers' joint income was verified by LC, not verified, or if the income source was verified",NaN,schema,NaN,False,NaN
79.0,None,* Employer Title replaces Employer Name for all loans listed after 9/23/2013,NaN,schema,NaN,False,NaN
78.0,None,None,NaN,schema,NaN,False,NaN
19.0,id,A unique LC assigned ID for the loan listing.,NaN,schema,NaN,False,NaN
14.0,fico_range_low,The lower boundary range the borrower’s FICO at loan origination belongs to.,NaN,schema,NaN,False,NaN
13.0,fico_range_high,The upper boundary range the borrower’s FICO at loan origination belongs to.,NaN,schema,NaN,False,NaN
24.0,is_inc_v,"Indicates if income was verified by LC, not verified, or if the income source was verified",NaN,schema,NaN,False,NaN
71.0,total_rev_hi_lim,Total revolving high credit/credit limit,NaN,schema,NaN,False,NaN


In [67]:
data_dict.loc[data_dict['LoanStatNew'] == 'term', 'target_type'] ='categorical'

In [68]:
data_dict.loc[data_dict['LoanStatNew'] == 'sub_grade', 'target_type'] = 'categorical'

In [69]:
data_dict.loc[data_dict['LoanStatNew'] == 'total_rec_late_fee', 'target_type'] = 'int64'

In [70]:
data_dict.loc[data_dict['LoanStatNew'] == 'total_acc', 'target_type'] = 'int64'

In [71]:
data_dict.loc[data_dict['LoanStatNew'] == 'open_il_24m', 'target_type'] = 'int64'

In [72]:
data_dict.loc[data_dict['LoanStatNew'] == 'open_il_6m', 'target_type'] = 'int64'

In [73]:
data_dict.loc[data_dict['LoanStatNew'] == 'acc_now_delinq', 'target_type'] = 'int64'

Create an int version fro emp_length:

In [111]:
df['emp_length_int'] = df['emp_length']\
    .str.replace('< 1 year', str(0))\
    .str.extract(r'(?P<years>[0-9]{1,2})')\
    .fillna(0)\
    .astype(np.int8)

In [116]:
data_dict.loc[data_dict['LoanStatNew'] == 'emp_length', 'target_type'] = 'int8'
data_dict.loc[data_dict['LoanStatNew'] == 'emp_length', 'changed'] = True

earliest_cr_line:
- Convert into date
- Calculate time gone by until current date in months
- Fix negatives values

In [172]:
df['months_since_earliest_cr_line'] = np.round(
( pd.to_datetime(CURRENT_DATE) -
    df.assign(year=df['earliest_cr_line'].str.extract(r'(?P<year>[0-9]{2})$').astype(np.int8).map(lambda y: 2000 + y if 0 <= y <= 17 else 1900 + y),
              month=df['earliest_cr_line'].str.extract(r'(?P<month>[A-Z]{1}[a-z]{2})'))\
       .apply(lambda d: datetime.strptime(d['month'] + '-' + str(d['year']), '%b-%Y'), axis=1)
  ) / np.timedelta64(30, 'D')
).astype('int64')


In [173]:
data_dict.loc[data_dict['LoanStatNew'] == 'earliest_cr_line', 'target_type'] = 'int64'
data_dict.loc[data_dict['LoanStatNew'] == 'earliest_cr_line', 'changed'] = True

Create an int version for term

In [175]:
df['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [179]:
df['term_int'] = df['term'].str.extract(r'(?P<year>[0-9]{1,2})').astype('int8')

In [180]:
data_dict.loc[data_dict['LoanStatNew'] == 'term', 'target_type'] = 'int8'
data_dict.loc[data_dict['LoanStatNew'] == 'term', 'changed'] = True

In [227]:
df = df.drop('emp_length', axis=1)

Calculate months since issuance:

In [183]:
df['months_since_issue_d'] = np.round(
( pd.to_datetime(CURRENT_DATE) -
    df.assign(year=df['issue_d'].str.extract(r'(?P<year>[0-9]{2})$').astype(np.int8).map(lambda y: 2000 + y if 0 <= y <= 17 else 1900 + y),
              month=df['issue_d'].str.extract(r'(?P<month>[A-Z]{1}[a-z]{2})'))\
       .apply(lambda d: datetime.strptime(d['month'] + '-' + str(d['year']), '%b-%Y'), axis=1)
  ) / np.timedelta64(30, 'D')
).astype('int64')

In [185]:
df['months_since_issue_d'].describe()

count    10000.00000
mean        74.79330
std          1.34928
min         73.00000
25%         74.00000
50%         75.00000
75%         76.00000
max         77.00000
Name: months_since_issue_d, dtype: float64

In [187]:
df['issue_date'] = df.assign(year=df['issue_d'].str.extract(r'(?P<year>[0-9]{2})$').astype(np.int8).map(lambda y: 2000 + y if 0 <= y <= 17 else 1900 + y),
                            month=df['issue_d'].str.extract(r'(?P<month>[A-Z]{1}[a-z]{2})'))\
                      .apply(lambda d: datetime.strptime(d['month'] + '-' + str(d['year']), '%b-%Y'), axis=1)

Convert into category:

In [239]:
df['verification_status'] = df['verification_status'].astype('category')
df['term'] = df['term'].astype('category')
df['sub_grade'] = df['sub_grade'].astype('category')
df['purpose'] = df['purpose'].astype('category')
df['grade'] = df['grade'].astype('category')
df['addr_state'] = df['addr_state'].astype('category')
df['application_type'] = df['application_type'].astype('category')
df['loan_status'] = df['loan_status'].astype('category')
df['home_ownership'] = df['home_ownership'].astype('category')
df['pymnt_plan'] = df['pymnt_plan'].astype('category')
df['initial_list_status'] = df['initial_list_status'].astype('category')

In [240]:
data_dict.loc[data_dict['LoanStatNew'] == 'grade', 'target_type'] = 'categorical'
data_dict.loc[data_dict['LoanStatNew'] == 'addr_state', 'target_type'] = 'categorical'
data_dict.loc[data_dict['LoanStatNew'] == 'application_type', 'target_type'] = 'categorical'
data_dict.loc[data_dict['LoanStatNew'] == 'loan_status', 'target_type'] = 'categorical'
data_dict.loc[data_dict['LoanStatNew'] == 'home_ownership', 'target_type'] = 'categorical'
data_dict.loc[data_dict['LoanStatNew'] == 'pymnt_plan', 'target_type'] = 'categorical'
data_dict.loc[data_dict['LoanStatNew'] == 'initial_list_status', 'target_type'] = 'categorical'

In [245]:
df['emp_title'].unique()

array([nan, 'Ryder', 'AIR RESOURCES BOARD', ..., 'Silicon Valley Bank',
       'M. S.', 'Sheraton agoura hills'], dtype=object)

Zip code to int

In [243]:
df['zip_code'] = df['zip_code'].str.extract(r'(?P<zip>[0-9]{3})').astype('int16')
data_dict.loc[data_dict['LoanStatNew'] == 'zip_code', 'target_type'] = 'int16'

In [244]:
df.memory_usage(deep=True)

Index                              80000
member_id                          80000
loan_amnt                          80000
funded_amnt                        80000
funded_amnt_inv                    80000
term                               10226
int_rate                           80000
installment                        80000
grade                              10650
sub_grade                          12857
emp_title                         651605
home_ownership                     10270
annual_inc                         80000
verification_status                10290
issue_d                           550000
loan_status                        10728
pymnt_plan                         10158
url                              1164373
desc                             2388956
purpose                            11323
title                             653983
zip_code                           20000
addr_state                         13367
dti                                80000
delinq_2yrs     